In [ ]:
class SingleChoiceQuestion(Question):
    """
    Class for a single-choice survey question.
    Handles:
    - Basic single-choice answers (stored as numbers).
    - Additional text responses if applicable (found dynamically).
    DE2, DE4, DE5, DE6, DE8, DE12, DE14, DE15, DE16, DE17, DE18, DE19, DE20, DE21, PL5, PL8,
    """
    def __init__(
        self,
        question_id: str,
        df: pd.DataFrame,
        df_raw: pd.DataFrame,
        value_transform: Callable[[Any], Any] = None,
        unit_hint: Optional[str] = None,
    ):
        super().__init__(question_id, df, df_raw, value_transform)
        self.unit_hint = unit_hint

        # Clean up responses
        self.responses = pd.to_numeric(self.df[question_id], errors="coerce").dropna()

        # Optional: collect additional free text responses (like for "Other")
        self.extra_texts = {}
        for col in df.columns:
            if col.startswith(f"{question_id}_") and col.endswith("_TEXT"):
                option_number = col.split("_")[1]
                text_series = df[col].dropna().astype(str)
                if not text_series.empty:
                    self.extra_texts[option_number] = text_series

    def __repr__(self):
        if self.responses.empty:
            return f"{self.question_text} – No responses provided."
        return f"{self.question_text} – {len(self.responses)} responses collected."

    def plot_distribution(self, display=True):
        value_counts = self.responses.value_counts().sort_index()
        total = len(self.responses)
        percentages = (value_counts / total * 100).round(2)

        # Decode value_map (gender 1 = woman, etc.)
        labels = [
            self.value_map.get(str(int(x)), f"Option {int(x)}")
            for x in value_counts.index
        ]

        print(f"{total} respondents out of {self.df.shape[0]} participants provided a response.")

        fig = self._plot_bar_distribution(
            labels=labels,
            percentages=percentages,
            title=self.question_text,
        )

        if display:
            fig.show()
        return fig


        """
        def __init__(self, question_id: str, df: pd.DataFrame, value_map: dict = None, value_transform: Callable[[Any], Any] = None, unit_hint: Optional[str] = None):
        super().__init__(question_id, df, df_raw, value_transform)
        #self.response = None  # Numeric response
        #self.extra_texts = {}  # Dictionary storing any additional text 
        #self.value_map = value_map or {}
        #self.value_transform = value_transform
        self.unit_hint = unit_hint
        
        participant_data = df.loc[df["responseId"] == response_id]

        if not participant_data.empty and question_id in participant_data.columns:
            self.response = str(participant_data[question_id].values[0])

        for col in participant_data.columns:
            if col.startswith(f"{question_id}_") and col.endswith("_TEXT"):
                option_number = col.split("_")[1]

                if option_number == self.response:  
                    text_value = participant_data[col].values[0]
                    if isinstance(text_value, list) or isinstance(text_value, np.ndarray):
                        if len(text_value) > 0:
                            text_value = text_value[0]

                    if pd.notna(text_value): 
                        self.extra_texts[option_number] = text_value.strip()

        if (
            self.value_transform 
            and self.unit_hint 
            and self.response in self.extra_texts
        ):
            try:
                numeric_val = float(self.extra_texts[self.response])
                transformed_val = self.value_transform(numeric_val, self.unit_hint)
                self.extra_texts[self.response] = str(transformed_val)
            except (ValueError, TypeError):
                pass  # gracefully fall back if parsing or transformation fails


        def __repr__(self):
            if self.responses.empty:
                return f"{self.question_text} – No responses provided."
            return f"{self.question_text} – {len(self.responses)} responses collected."
   


    def plot_distribution(self):
        if self.df[self.question_id].isnull().all():
            print("No valid response data available.")
            return

        responses = pd.to_numeric(self.df[self.question_id], errors="coerce").dropna()
        if responses.empty:
            print("No responses to plot.")
            return

        total_respondents = len(responses)
        total_participants = self.df.shape[0]
        print(f"{total_respondents} respondents out of {total_participants} participants provided a response.")

        if self.value_map:
            mapped = responses.map(lambda x: self.value_map.get(str(int(x)), f"Option {int(x)}"))
        else:
            mapped = responses.astype(str)

        value_counts = mapped.value_counts()
        percentages = (value_counts / total_respondents * 100).round(2)

        # Optional: preserve original value_map order
        if self.value_map:
            label_order = [self.value_map[k] for k in sorted(self.value_map.keys(), key=int)]
            percentages = percentages.reindex(label_order).dropna()

        fig = self._plot_bar_distribution(
            labels=percentages.index,
            percentages=percentages.values,
            title=self.question_text
        )

        fig.show()
        return fig"""